In [2]:
import boto3
client = boto3.client('textract')

In [36]:
response = client.start_document_analysis(
    DocumentLocation={
        'S3Object': {
            'Bucket': 'fairstone',
            'Name': 'docqna/CaseFund/CaseFund.pdf',     
        }
    },
    FeatureTypes=[
        'LAYOUT',"TABLES"
    ],
    ClientRequestToken='cdddssdatkl',
    # JobTag='string',
    # NotificationChannel={
    #     'SNSTopicArn': 'string',
    #     'RoleArn': 'string'
    # },
    OutputConfig={
        'S3Bucket': 'fairstone',
        # 'S3Prefix': 'string'
    },
    # KMSKeyId='string',
   
)
response

{'JobId': '3fa3a6f711bdd6bf30aad6b41876a202de7e679eb3a5d255d07eecd10f3423ab',
 'ResponseMetadata': {'RequestId': '004b4bfe-1d08-4394-aa45-28e0955db87d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '004b4bfe-1d08-4394-aa45-28e0955db87d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '76',
   'date': 'Mon, 22 Jan 2024 00:19:59 GMT'},
  'RetryAttempts': 0}}

In [ ]:
responses1 = client.get_document_analysis(
    JobId=response['JobId']#'aa594af8fc71e7155b18610179accad70aa74f62d12ba933ccbfe9b548912c99',
 
)
responses1

In [38]:
def merge_consecutive_runs_refactored(number_list):
    merged_list = [number_list[i] if i == 0 or number_list[i] != number_list[i-1] + 1 else None
                   for i in range(len(number_list))]

    return [num for num in merged_list if num is not None]

In [39]:
import os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint


def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    scores = []
    merged_cells = []
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'MERGED_CELL':
            merged_cells.extend(blocks_map[relationship['Ids'][0]]['Relationships'][0]['Ids'])
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                    
                    # get confidence score
                    scores.append(str(cell['Confidence']))
                        
                    # get the text value
                    rows[row_index][col_index] = {"text":get_text(cell, blocks_map),"ids":child_id}
                    # rows[row_index][col_index]={}
    return rows, scores, merged_cells


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        if "," in word['Text'] and word['Text'].replace(",", "").isnumeric():
                            text += '"' + word['Text'] + '"' + ' '
                        else:
                            text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '
    return text


def get_table_csv_results(response, blocks_map=None):
    # Get the text blocks
    blocks=response['Blocks']
    # pprint(blocks)
    if not blocks_map:
        blocks_map = {}
    table_blocks = []
    for block in blocks:
        if not blocks_map:
            blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"
    
    rwss=[]
    merged_cellss=[]
    csv_text=[]
    csv = ''
    for index, table in enumerate(table_blocks):
        word, rws,merged_cells=generate_table_csv(table, blocks_map, index +1)
        csv += word
        csv += '\n\n'
        rwss.append(rws)
        merged_cellss.append(merged_cells)
        csv_text.append(csv)
        
    return csv_text, rwss, merged_cellss

def generate_table_csv(table_result, blocks_map, table_index):
    rows, scores, merged_cells = get_rows_columns_map(table_result, blocks_map)
    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        for col_index, text in cols.items():
            col_indices = len(cols.items())
            csv += '{}'.format(text['text']) + ","
        csv += '\n'
        
    csv += '\n\n Confidence Scores % (Table Cell) \n'
    cols_count = 0
    for score in scores:
        cols_count += 1
        csv += score + ","
        if cols_count == col_indices:
            csv += '\n'
            cols_count = 0

    csv += '\n\n\n'
    return csv, rows, merged_cells 

def csv_creator(file_name, blocks_map=None):
    result=get_table_csv_results(file_name, blocks_map)
    if "NO Table FOUND" in result:
        return ""
    else:
        table_csv, rows,merged_cells=result[0],result[1],result[2]
        page_list=[] 
        table_string=''

        table_list=[]
        for items in rows:
            table_dict={}
            for row_index, cols in items.items():
                table_dict[row_index]=[]
                for col_index, text in cols.items():    

                    col_indices = len(cols.items())
                    if text['ids'] in merged_cells:
                        table_string+=text["text"]
                        if table_dict[row_index]:
                            table_dict[row_index].pop(-1)
                        table_dict[row_index].append(table_string)
                    else:            
                        table_string += '{}'.format(text['text']) + ","
                        table_dict[row_index].append(text["text"])
                
                table_string += '\n'
            table_list.append(table_dict)
            cell_table=[x for x in [blocks_map[x] for x in blocks_map if blocks_map[x]['BlockType']=="TABLE"] if text['ids']in x['Relationships'][0]['Ids']]
            page=cell_table[0]['Page']
            page_list.append(page)
        import pandas as pd
        ids = []
        for items in rows:
            for v1 in items.values():
                for v2 in v1.values(): 
                    ids.append(v2['ids'])
        
        header=""
        page=""
        df_list=[]
        header_list=[]
  
        # Check if the first row's index size mismatches the rest of the rows
        for table_dict in table_list:

            if len(table_dict.keys())>1:
                if len(table_dict.get(1, [])) != len(table_dict.get(2, [])):
                    header=table_dict.pop(1)
                    columns = table_dict.pop(2)
                # Extract the column names from the first row
                else:
                    if any(table_dict[1]):
                        columns = table_dict.pop(1)
                    else:
                        columns = table_dict.pop(2)
           
                # Convert the dictionary to a DataFrame
                try:
                    df = pd.DataFrame.from_dict(table_dict, orient='index', columns=columns)
                    for page in set([blocks_map[x]["Page"] for x in ids]):
                        pages=page 
                except:
                    df = pd.DataFrame.from_dict({}, orient='index', columns=columns)
                    for page in set([blocks_map[x]["Page"] for x in ids]):
                        pages=page 
            else:
                columns = table_dict.pop(1)
                df = pd.DataFrame.from_dict({}, orient='index', columns=columns)
                for page in set([blocks_map[x]["Page"] for x in ids]):
                        pages=page 
            df_list.append(df)
            header_list.append(header) 
    
        return df_list, header_list, page_list,rows


In [40]:
import boto3

def extract_words_in_bounding_box(textract_response, holder):
    countt=0
    words_in_box = {}
    page=-1
    words=[]
    ground_truth_lines=[]
    ground_truth_doc={}
    lines=[]
    table_lines={}
    entire_doc_lines={}
    item_type=[]
    for item in holder:        
        target_page=item['Page']
        bounding_box=item['BoundingBox']
        layout_type=item['Type']
        item_type.append(layout_type)
        comptroller=1
        orchestrator=1
        g_orchestrator=1
        if page!= item['Page']:
            words=[]
            lines=[]
            ground_truth_lines=[]
        page=item['Page']
        # print(page)
        for block in textract_response['Blocks']:
            if  (
                block['BlockType'] == 'LINE' and
                'Page' in block and
                block['Page'] == target_page and
                'Geometry' in block 
            ):
                box = block['Geometry']['BoundingBox']
                # print(block['Page'])

                # Check if the word's bounding box is within the specified bounding box
                if (
                    bounding_box['Left']-0.015*bounding_box['Left'] <= box['Left'] and
                    bounding_box['Top']-0.015*bounding_box['Top'] <= box['Top'] and
                    (bounding_box['Left'] + bounding_box['Width'])+0.015*(bounding_box['Left'] + bounding_box['Width']) >= box['Left'] + box['Width'] and
                    (bounding_box['Top'] + bounding_box['Height'])+0.015*(bounding_box['Top'] + bounding_box['Height']) >= box['Top'] + box['Height']
                ):
                    # print(box['Left'],bounding_box['Left'])
                    # words_in_box.append(block['Text'])
                    lines.append(block['Text'])
                    entire_doc_lines[block['Page']]=lines
                    
                    if layout_type== 'LAYOUT_TABLE' and g_orchestrator==1:
                        ground_truth_lines.append("<table>")
                        g_orchestrator=0
                    if len(item_type)>1 and item_type[-1]!='LAYOUT_TABLE' and item_type[-2]=='LAYOUT_TABLE' and g_orchestrator==1:
                        ground_truth_lines.append("</table>")
                        g_orchestrator=0
                    ground_truth_lines.append(block['Text'])
                    ground_truth_doc[block['Page']]=ground_truth_lines
                    
                    
                if layout_type !='LAYOUT_TABLE':
                    box = block['Geometry']['BoundingBox']
                # print(block['Page'])

                # Check if the word's bounding box is within the specified bounding box
                    if (
                        bounding_box['Left']-0.015*bounding_box['Left'] <= box['Left'] and
                        bounding_box['Top']-0.015*bounding_box['Top'] <= box['Top'] and
                        (bounding_box['Left'] + bounding_box['Width'])+0.015*(bounding_box['Left'] + bounding_box['Width']) >= box['Left'] + box['Width'] and
                        (bounding_box['Top'] + bounding_box['Height'])+0.015*(bounding_box['Top'] + bounding_box['Height']) >= box['Top'] + box['Height']
                    ):
                        # print(box['Left'],bounding_box['Left'])
                        # words_in_box.append(block['Text'])
                        words.append(block['Text'])
                        words_in_box[block['Page']]=words
                if layout_type =='LAYOUT_TABLE':
                    # orchestrator=comptroller
                    if comptroller ==1:
                        countt+=1
                        words.append("<table>")
                        # words.append(table)
                        words_in_box[block['Page']]=words
                        comptroller=0
                    
                    box = block['Geometry']['BoundingBox']
                    # print(block['Page'])

                    # Check if the word's bounding box is within the specified bounding box
                    if (
                        bounding_box['Left']-0.015*bounding_box['Left'] <= box['Left'] and
                        bounding_box['Top']-0.015*bounding_box['Top'] <= box['Top'] and
                        (bounding_box['Left'] + bounding_box['Width'])+0.015*(bounding_box['Left'] + bounding_box['Width']) >= box['Left'] + box['Width'] and
                        (bounding_box['Top'] + bounding_box['Height'])+0.015*(bounding_box['Top'] + bounding_box['Height']) >= box['Top'] + box['Height']
                    ):
                        if block['Page'] in table_lines:
    #                     # # print(table_lines)
                            if orchestrator==1:
                                if not (block['Text'] in table_lines[block['Page']] or block['Text']+" xxxxxx" in table_lines[block['Page']]):
                                    table_lines[block['Page']].extend([block['Text']+" xxxxxx"])
                                    # print(block['Page'],[block['Text']+" xxxxxxORCH"])
                                    orchestrator=0
                            else:
                                # continue
                                if not block['Text'] in table_lines[block['Page']]:
                                    table_lines[block['Page']].extend([block['Text']])
                        else:
                        # continue
                            table_lines[block['Page']]=[block['Text']+" xxxxxx"]
                            # print(block['Page'],[block['Text']+" xxxxxx"])
                            orchestrator=0

    return words_in_box, table_lines,entire_doc_lines, ground_truth_doc#'\n'.join(words_in_box)


In [41]:

responses1 = client.get_document_analysis(
    JobId=response['JobId'], 
)
geom=[]
blocks_map = {}
doc=[]
words_only=[]
table_holder={}
table_lines={}
non_layout_table_holder={}
ground_truth_doc={}
next_tokens=[]
complete_response_list=[]
for block in responses1['Blocks']:
    holder={}
    if block['BlockType'] in [
        # 'LAYOUT_FIGURE',
 # 'LAYOUT_FOOTER',
 'LAYOUT_HEADER',
 'LAYOUT_PAGE_NUMBER',
 'LAYOUT_SECTION_HEADER',
 'LAYOUT_TEXT',
 'LAYOUT_TITLE',
         'LAYOUT_TABLE',
       
                             ]:
        holder['BoundingBox']=block['Geometry']['BoundingBox']
        holder["Page"]=block['Page']
        holder["Type"]=block['BlockType']
        geom.append(holder)
        # print(x)
    
    blocks_map[block['Id']] = block

        

if "NextToken" in responses1.keys():
    next_token=responses1['NextToken']
    while next_token:
        responses1 = client.get_document_analysis(
                    JobId=response['JobId'],
                    # MaxResults=123,
                    NextToken=next_token,
                    )
        for block in responses1['Blocks']:
            holder={}
            if block['BlockType'] in [
                    # 'LAYOUT_FIGURE',
                     # 'LAYOUT_FOOTER',
                     'LAYOUT_HEADER',
                     'LAYOUT_PAGE_NUMBER',
                     'LAYOUT_SECTION_HEADER',
                     'LAYOUT_TEXT',
                     'LAYOUT_TITLE',
               
                 'LAYOUT_TABLE',]:
                holder['BoundingBox']=block['Geometry']['BoundingBox']
                holder["Page"]=block['Page']
                holder["Type"]=block['BlockType']
                geom.append(holder)
            blocks_map[block['Id']] = block
        if "NextToken" in responses1.keys():
            next_token=responses1['NextToken']
        else:
            next_token=""
            break

responses1 = client.get_document_analysis(
    JobId=response['JobId'],
    # MaxResults=123,
    # NextToken=nt,
)
result = extract_words_in_bounding_box(responses1, geom)
doc.append(result[0])
table_lines.update(result[1])
non_layout_table_holder.update(result[2])
ground_truth_doc.update(result[-1])

table_result=csv_creator(responses1,blocks_map)
if table_result:
    table, header,pages=table_result[0], table_result[1],table_result[2]
    
    for ids,page in enumerate(pages):
        doc
        if page in table_holder:
            table_holder[page].extend([header[ids]]+[table[ids]])
        else:            
            dummy=[]
            dummy.extend([header[ids]]+[table[ids]])
            table_holder[int(page)]= dummy
if "NextToken" in responses1.keys():
    next_token=responses1['NextToken']           
    while next_token:
        responses1 = client.get_document_analysis(
            JobId=response['JobId'],
            # MaxResults=123,
            NextToken=next_token,
        )

        table_result=csv_creator(responses1,blocks_map)
        if table_result:
            table, header,pages=table_result[0], table_result[1],table_result[2]

            for ids,page in enumerate(pages):
                if page in table_holder:
                    table_holder[page].extend([header[ids]]+[table[ids]])
                else:      
                    dummy=[]
                    dummy.extend([header[ids]]+[table[ids]])
                    table_holder[int(page)]= dummy

        # print(next_token, end='\n')    
        next_tokens.append(next_token)
        result = extract_words_in_bounding_box(responses1, geom)    
        doc.append(result[0])
        table_lines.update(result[1])
        non_layout_table_holder.update(result[2])   
        ground_truth_doc.update(result[-1])      
        if "NextToken" in responses1.keys():
            next_token=responses1['NextToken']
        else:
            next_token=""
            break

for d in doc:
    for k, v in d.items():
        layout_table_count=v.count('<table>')
        if k in table_holder.keys():
            non_layout_table_count=len(table_holder[k])/2 # Table_holder has header and tables in list 
        else:
            non_layout_table_count=0
        if layout_table_count == non_layout_table_count:
            continue
        elif non_layout_table_count==0:
            table_index_posts=[index for index, element in enumerate(v) if element == '<table>']
            merged_list_refactored = merge_consecutive_runs_refactored(table_index_posts)

            for ids in merged_list_refactored:
                #index of all tables in extracted text
                dynamic_table_index_posts=[index for index, element in enumerate(v) if element == '<table>']
                # handle consecutive tables series
                dynamic_merged_list_refactored= merge_consecutive_runs_refactored(dynamic_table_index_posts)
                table_index_pos=v.index("<table>")  
                if table_index_pos+1 in dynamic_table_index_posts: # if consecutive tables
                    if len(dynamic_merged_list_refactored)>1:
                        next_non_consecutive_item=dynamic_merged_list_refactored[dynamic_merged_list_refactored.index(table_index_pos)+1]
                        last_consecutive_series_item=dynamic_table_index_posts[dynamic_table_index_posts.index(next_non_consecutive_item)-1]
                    else:
                        last_consecutive_series_item=dynamic_table_index_posts[-1]

                    if dynamic_merged_list_refactored[0]!=len(v): #Check that table in text is not the last item
                        # index for the word trailing the table, handle for multiple occurence of that word- in the doc
                        index_stopper=[i for i in range(len(non_layout_table_holder[2]) - 1) if non_layout_table_holder[2][i] == v[last_consecutive_series_item+1] and non_layout_table_holder[2][i + 1] ==  v[last_consecutive_series_item+2]][0]

                        replace_text=non_layout_table_holder[k][table_index_pos-1:non_layout_table_holder[k].index(v[last_consecutive_series_item+1],index_stopper)][1:]
                        v[table_index_pos:last_consecutive_series_item+1]=replace_text
                    else:
                        replace_text=non_layout_table_holder[k][table_index_pos-1:][1:]
                        v[table_index_pos:last_consecutive_series_item+1]=replace_text
                else:  # No consecutive table series
                    replace_text=non_layout_table_holder[k][non_layout_table_holder[k].index(v[table_index_pos-1]):non_layout_table_holder[k].index(v[table_index_pos+1])][1:]
                    v[table_index_pos:table_index_pos+1]=replace_text
        elif non_layout_table_count>layout_table_count:
            non_layout_page_word_count=[]
            for i in range(layout_table_count*2):
                non_layout_page_word_count.extend(table_holder[k][i])
            length_of_non_layout_page=len(" ".join(non_layout_page_word_count).split())+len(" ".join(v).split())-layout_table_count
            length_of_layout_page=len(" ".join(non_layout_table_holder[k]).split())
            if length_of_layout_page>length_of_non_layout_page and (abs(length_of_layout_page-length_of_non_layout_page)/max(length_of_layout_page,length_of_non_layout_page))>0.2:
                v.insert(v.index("<table>")+1,"<table>")

table_count_list={}
for d in doc:
    for k, v in d.items():
        table_count_list[k]=v.count('<table>')


table_count_lists = []
for k, v in table_count_list.items():
    if v != 0:
        table_count_lists.extend([k]*v)

doc_layout_extract={}  
if not len(table_count_lists) or  not len([x["Page"] for x in  geom if x['Type']=="LAYOUT_TABLE"]):
    print("NO TABLES FOUND")
    for item in doc:
        for k, v in item.items():
            doc_layout_extract[k]=v
else:
    for d in doc:
        for k, v in d.items():
            if v.count("<table>")>1:
                txt=v
                ids=0        
                for count in range(v.count("<table>")):                                
                    header_and_table=f"{table_holder[k][ids]}\n<tables>{table_holder[k][ids+1].to_csv(index=False, sep='|')}</tables>"
                    if count+1<= table_lines[k].count("xxxxxx"):
                        header_before_table=[x for x in table_lines[k] if "xxxxxx" in x][count].split("xxxxxx")[0]
                        if not header_before_table in header_and_table:
                            header_and_table=f"{header_before_table}\n{header_and_table}"               
                    table_holder_index=txt.index("<table>")
                    txt[table_holder_index]=header_and_table
                    ids+=2
                doc_layout_extract[k]=txt
            elif v.count("<table>")==1:
                txt=v
                header_and_table=f"{table_holder[k][0]}\n<tables>{table_holder[k][1].to_csv(index=False, sep='|')}</tables>"
                header_before_table=[x for x in table_lines[k] if "xxxxxx" in x][0].split("xxxxxx")[0]
                if not header_before_table in header_and_table:
                    header_and_table=f"{header_before_table}\n{header_and_table}"
                table_holder_index=txt.index("<table>")
                txt[table_holder_index]=header_and_table            
                doc_layout_extract[k]=txt
            else:
                doc_layout_extract[k]=v
    for k, v in ground_truth_doc.items():

        page_table_idx=[doc_layout_extract[k].index(x) for x in doc_layout_extract[k] if "<tables>" in x]
        ground_truth_table_ids=[ground_truth_doc[k].index(x) for x in ground_truth_doc[k] if "<table>" in x]
        if page_table_idx:
            for table_count,table_idx in enumerate(ground_truth_table_ids):
                page_tab=doc_layout_extract[k][page_table_idx[table_count]]
                page_tab_to_list=page_tab.split("\n")
                ground_truth_table_header=ground_truth_doc[k][table_idx+1]
                page_tab_headers=page_tab_to_list[0] if page_tab_to_list[0] else page_tab_to_list[1]
                if ground_truth_table_header:
                    found = ground_truth_table_header in page_tab_headers #any(result_table_header in item for item in tab_headers)
                    if found:
                        continue
                    else:
                        doc_layout_extract[k][page_table_idx[table_count]]=f"{ground_truth_table_header}\n"+page_tab
                else:
                    continue


W4dkHKDMY/+8CbPrG83hEDA1iPUg6xkZrjLIk5Pxc+iUV0b+VUNCVivn3YTCDxr7umdyJbHfnUf667qQC9otaCJDgOO1lCpWeTT6dsuw3rJbICkKbgCyheCu4VF43ZX8MXHFd2I=
W4dkHKDMY/+8CbPrG83hEDA1iPUg6xkZrjLIk5Pxc+iUV0b+VUNCVivn3YTCDxr7umdyJbHfnUf667qQC9otaCJDgOO1lCpWeTT6dsu21rxbIykKblNTGXaKxG2KNkO76E4vKrc=
W4dkHKDMY/+8CbPrG83hEDA1iPUg6xkZrjLIk5Pxc+iUV0b+VUNCVivn3YTCDxr7umdyJbHfnUf667qQC9otaCJDgOO1lCpWeTT6dsu63rtbIikKbsPGC/uiWbGl2+3HzvUbJlU=
W4dkHKDMY/+8CbPrG83hEDA1iPUg6xkZrjLIk5Pxc+iUV0b+VUNCVivn3YTCDxr7umdyJbHfnUf667qQC9otaCJDgOO1lCpWeTT6dsyx1rtbJSkKbjyq897elbsDPfaOoX28SwE=
W4dkHKDMY/+8CbPrG83hEDA1iPUg6xkZrjLIk5Pxc+iUV0b+VUNCVivn3YTCDxr7umdyJbHfnUf667qQC9otaCJDgOO1lCpWeTT6dsy22LpbJCkKbklFQtWjJLWnzmIxR22tYw0=
W4dkHKDMY/+8CbPrG83hEDA1iPUg6xkZrjLIk5Pxc+iUV0b+VUNCVivn3YTCDxr7umdyJbHfnUf667qQC9otaCJDgOO1lCpWeTT6dsy72L5bJykKbrupl6UPoYVssk1zheY4fM4=
W4dkHKDMY/+8CbPrG83hEDA1iPUg6xkZrjLIk5Pxc+iUV0b+VUNCVivn3YTCDxr7umdyJbHfnUf667qQC9otaCJDgOO1lCpWeTT6ds2x3b1bJikKbmUkzXk1PDnlJZW1wKQ42kw=
W4dkHKDMY/+8CbPrG83hEDA1iPUg6xkZrjLIk5Pxc

In [ ]:
# dictionary containing extracted document per page
doc_layout_extract

In [42]:
import re
import pandas as pd
from io import StringIO

max_words = 200
chunks = {}
entire_list_chunk=[]
overlap=50
for page, lines in doc_layout_extract.items():
    page_chunks = []
    current_chunk = []
    num_words = 0   
    start_page=0
    for line in lines:
                   
            
        if len(current_chunk)<2 and entire_list_chunk and start_page==0:    
            line_word_count=0
            lines_length=len(entire_list_chunk[-1])
            for ids, rows in enumerate(entire_list_chunk[-1]):
                last_items_up=entire_list_chunk[-1][lines_length-1-ids]
                current_chunk.insert(0,last_items_up) 
                line_word_count+=len(re.findall(r'\w+', last_items_up)) 
                if line_word_count+ len(re.findall(r'\w+', entire_list_chunk[-1][max(0,lines_length-2-ids)])) >overlap: 
                    break
            num_words=line_word_count
            start_page=1
            
            
        next_num_words = num_words + len(re.findall(r'\w+', line))   
        
        if next_num_words > max_words:# and "<table>" not in line:
            page_chunks.append(current_chunk)
            entire_list_chunk.append(current_chunk)
            current_chunk = []
            num_words = 0
        
        if "<tables>" in line:
            
            header=line.split("<tables>")[0]

            table = line.split("<tables>")[-1].split("</tables>")[0]
            df = pd.read_csv(StringIO(table), sep='|')
            # if 'Unnamed: 0' in df.columns.to_list():
            #     df = pd.read_csv(StringIO(table), sep='|').drop('Unnamed: 0',axis=1)
 
                

            table_chunks = []
            curr_chunk = [df.columns.to_list()]
            words = 0

            for row in df.itertuples():
                curr_chunk.append(row) 
                words += len(re.findall(r'\w+', str(row)))
                # print(words)

                if words > max_words:
                    table_chunks.append("\n".join(["| ".join(str(x) for x in curr_chunk[0])] + ["| ".join(str(x) for x in r) for r in curr_chunk[1:]]))
                    # curr_chunk = [curr_chunk[0]]
                    words = len(re.findall(r'\w+', str(curr_chunk[-1])))
                    tab_chunk="\n".join(["| ".join(str(x) for x in curr_chunk[0])] + ["| ".join(str(x) for x in r) for r in curr_chunk[1:]])
                    current_chunk.extend([header]+[tab_chunk])
                    page_chunks.append(current_chunk)
                    entire_list_chunk.append([header]+table_chunks)
                    num_words=0
                    current_chunk=[]
                    curr_chunk = [curr_chunk[0]]

            if curr_chunk != [df.columns.to_list()]:
                table_chunks.append("\n".join(["| ".join(str(x) for x in curr_chunk[0])] + ["| ".join(str(x) for x in r) for r in curr_chunk[1:]]))
                tab_chunk="\n".join(["| ".join(str(x) for x in curr_chunk[0])] + ["| ".join(str(x) for x in r) for r in curr_chunk[1:]])
                         
                current_chunk.extend([header]+[tab_chunk])
                page_chunks.append(current_chunk)
                entire_list_chunk.append([header]+table_chunks)
                num_words=0
                current_chunk=[]
        
        if  "<tables>" not in line:
            current_chunk.append(line)    
            num_words += len(re.findall(r'\w+', line))
            # if page==14:
            #     print(current_chunk)

    if current_chunk:
        page_chunks.append(current_chunk)
        entire_list_chunk.append(current_chunk)
    chunks[page] = page_chunks

In [43]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

domain_endpoint = "search-rag2-fj5bhrqjlj7bhezstpki46uhlm.us-east-1.es.amazonaws.com"
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, "us-east-1", service, session_token=credentials.token)
os_ = OpenSearch(
    hosts = [{'host': domain_endpoint, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    timeout=120,        
    # http_compress = True, # enables gzip compression for request bodies
    connection_class = RequestsHttpConnection
)

mapping = {
  'settings': {
    'index': {  
      'knn': True,
      "knn.algo_param.ef_search": 64,            
    }
      },

      'mappings': {  
        'properties': {
          'embedding': {
            'type': 'knn_vector', 
            'dimension': 384,
            "method": {
              "name": "hnsw",       
              "space_type": "l2",
              "engine": "lucene",
              "parameters": {
                 "ef_construction": 72,
                 "m":  72
               }
            }
          },

          'passage_id': {
            'type': 'keyword'
          },

          'passage': {
            'type': 'text'
          },

          'doc_id': {
            'type': 'keyword'
          },
        
          'table': {
            'type': 'text'
          },

        }
      }
    }
# st.write(mapping)
domain_index = f"experiment5"    

if not os_.indices.exists(index=domain_index):        
    os_.indices.create(index=domain_index, body=mapping)
    # Verify that the index has been created
    if os_.indices.exists(index=domain_index):
        print(f"Index {domain_index} created successfully.")
    else:
        print(f"Failed to create index '{domain_index}'.")
else:
    print(f'{domain_index} Index already exists!')

i = 1
import boto3
SAGEMAKER=boto3.client('sagemaker-runtime')
for page, chunkks in chunks.items(): # Iterate through dict with chunk page# and content
    chunk_id = page # take care of multiple chunks in same page (*) is used as delimiter
   
    
    for chunk in chunkks:
        passage_chunk="\n".join(chunk)
        payload = {'text_inputs': [passage_chunk]}
        payload = json.dumps(payload).encode('utf-8')

        response = SAGEMAKER.invoke_endpoint(EndpointName="jumpstart-dft-hf-textembedding-all-minilm-l6-v2-2x", 
                                                    ContentType='application/json',  
                                                    Body=payload)

        model_predictions = json.loads(response['Body'].read())
        embedding = model_predictions['embedding'][0]
        table=[]
        if page in table_holder:
            for ids in range(0,len(table_holder[page]),2):
                header=table_holder[page][ids]
                tsv=table_holder[page][ids+1].to_csv(index=False,sep='|')
                table.append(header)
                table.append(tsv)
            table="\n".join(table)
                
        document = { 
            'doc_id':"chevron-2021",
            'passage_id': chunk_id,
            'passage': passage_chunk, 
            'embedding': embedding,
            'table':table
        }
        try:
            response = os_.index(index=domain_index, body=document)
            i += 1
            # Check the response to see if the indexing was successful
            if response["result"] == "created":
                print(f"Document indexed successfully with ID: {response['_id']}")
            else:
                print("Failed to index document.")
        except RequestError as e:
            logging.error(f"Error indexing document to index '{domain_index}': {e}")
        

experiment5 Index already exists!
Document indexed successfully with ID: WFKKLo0ByVMZnPenRo57
Document indexed successfully with ID: pn-KLo0BBfj2gFS_RtWw
Document indexed successfully with ID: WVKKLo0ByVMZnPenR45k
Document indexed successfully with ID: p3-KLo0BBfj2gFS_SNUM
Document indexed successfully with ID: WlKKLo0ByVMZnPenSI6a
Document indexed successfully with ID: qH-KLo0BBfj2gFS_SNXy
Document indexed successfully with ID: W1KKLo0ByVMZnPenSY5k
Document indexed successfully with ID: qX-KLo0BBfj2gFS_SdXr
Document indexed successfully with ID: XFKKLo0ByVMZnPenSo4q
Document indexed successfully with ID: qn-KLo0BBfj2gFS_StWe
Document indexed successfully with ID: XVKKLo0ByVMZnPenS44n
Document indexed successfully with ID: q3-KLo0BBfj2gFS_S9WK
Document indexed successfully with ID: XlKKLo0ByVMZnPenS47n
Document indexed successfully with ID: rH-KLo0BBfj2gFS_TNVX
Document indexed successfully with ID: X1KKLo0ByVMZnPenTI7L
Document indexed successfully with ID: rX-KLo0BBfj2gFS_TdVV
Docume

In [ ]:
response= SAGEMAKER.invoke_endpoint(EndpointName="jumpstart-dft-hf-textembedding-all-minilm-l6-v2-2x", 
                                                    ContentType='application/json',  
                                                    Body=json.dumps({'text_inputs': ["FUNDV gross IRR?"]}))
model_predictions = json.loads(response['Body'].read())
embedding = model_predictions['embedding'][0]
query = {
    'size': 3,
    'query': {
        "knn": {
          "embedding": {
            "vector": embedding,
            "k": 3
          }
        }
      }
    }
response = os_.search(index=domain_index, body=query)
response